**<font color = black size=6>实验十三: Q-learning</font>**


**<font color = blue size=4>第一部分: Q-learning 算法介绍</font>**


**1) Q-learning 伪代码**:


![Q-learning.jpg](Q-learning.jpg)


初始化 Q-table  
每个 epoch 循环:  
&emsp;&emsp;初始化 state $S$  
&emsp;&emsp;每一个 step 循环:  
&emsp;&emsp;&emsp;&emsp;根据贪婪程度选择一个 action $A$  
&emsp;&emsp;&emsp;&emsp;开始进行 action $A$，观察得到的 reward $R$, 下一个 state $S^{'}$  
&emsp;&emsp;&emsp;&emsp;更新 Q-table 中的 Q($S$,$A$): Q($S$,$A$) = Q($S$,$A$) + $\alpha [R + \gamma \max_{a^{'}}Q(S^{'},a^{'}) - Q(S,A)]$  
&emsp;&emsp;&emsp;&emsp;将当前状态 $S$ 更新为 $S^{'}$  
&emsp;&emsp;直到$S$到达终点


In [13]:
import numpy as np
import pandas as pd

**2)迷宫**  
假设在某个 5x5 大小的迷宫中,玩家从起点(0,0)出发，中间包括四种地段：  
1.路径(正常通过，每次经过会获得-1 的奖励，游戏继续)；  
2.障碍(到达后游戏结束，到达会获得-5 的奖励)；  
3.陷阱(到达后游戏结束，到达会获得-15 的奖励)；  
4.终点(到达后游戏结束，到达会获得+25 的奖励)


我们用下列矩阵代表迷宫，S(0,0)代表起点，E(4,4)代表终点，T(1,1)代表陷阱,B(3,1)和(1,3)代表障碍，其余 0 代表路径  
[S,0,0,0,0],  
 [0,T,0,B,0],  
 [0,0,0,0,0],  
 [0,B,0,0,0],  
 [0,0,0,0,E],


Action: 每次玩家有四种行动方式（上，下，右，左），分别用 0，1，2，3 代表


Q-table: Q-learning 中最重要的即为其拥有的 Q-table，代表着该算法中的行为准则。每次进行行动前都需要根据 Q-table 来进行判断，每次算法学习即为更新 Q-table。


In [14]:
# 定义一个空的Q-table
action_space = ["up", "down", "left", "right"]
actions = len(action_space)
q_table = pd.DataFrame(columns=list(range(actions)), dtype=np.float64)
print(q_table)

Empty DataFrame
Columns: [0, 1, 2, 3]
Index: []


$\epsilon$-greedy: 因为在初始阶段, 随机的探索环境, 往往比固定的行为模式要好, 所以这也是累积经验的阶段, 我们希望探索者不会那么贪婪(greedy)。 所以 $\epsilon$ 就是用来控制贪婪程度的值。$\epsilon$ = 0.9 意味着 90% 的时间是选择最优策略, 10% 的时间来探索。  
$\gamma$ 是对未来 reward 的衰减值  
$\alpha$是学习率, 来决定这次的误差有多少是要被学习的, $\alpha$是一个小于 1 的数


In [15]:
gamma = 0.9  # 自己设置
alpha = 0.9  # 自己设置

环境反馈：当玩家从某个 state $s$通过某个 action $a$到达另一个 state $s\_$后，会获得一定的奖励$r$。而后我们就会根据 Q-learning 算法使用$(s,a,s\_,r)$更新 Q-table


例子：玩家在起点(0,0)，此时的 Q-table 如下所示：  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; action  
state&nbsp; &nbsp; &nbsp; 0 &nbsp; 1 &nbsp; 2 &nbsp; 3  
[0,0]&nbsp; &nbsp; &nbsp; 0 &nbsp; -1 &nbsp; 0 &nbsp; 0  
[0,1]&nbsp; &nbsp; &nbsp; 0 &nbsp; -1 &nbsp; 3 &nbsp; 2


In [16]:
q_table = q_table.append(
    pd.Series(
        [0, -1, 0, 0],
        index=q_table.columns,
        name=str([0, 0]),
    )
)
q_table = q_table.append(
    pd.Series(
        [0, -1, 3, 2],
        index=q_table.columns,
        name=str([0, 1]),
    )
)
print(q_table)

          0    1    2    3
[0, 0]  0.0 -1.0  0.0  0.0
[0, 1]  0.0 -1.0  3.0  2.0


C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\662793476.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  q_table = q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\662793476.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  q_table = q_table.append(


此时玩家的 state 为(0,0)  
他现在有两种行动方式，一种是向右(action = 2)，一种是向下(action = 1)。此时我们的贪婪程度为 0.9，所以我们随机生成一个 0 到 1 的浮点数。如果小于 0.9 则采用贪婪模式，即选择该 state 所对应的 action 中对应的 value 最大的一个 action；如果大于 0.9，则为探索模式，则我们从该 state 中的所有 action 中随机选取一个 action


In [17]:
s = [0, 0]
# 生成随机数a
# if a<epsilon:
# 寻找该state s中value最大的action作为其action
# else:
# 该state s中所有action中随机选一个作为其action

这里我们采用贪婪模式，查询 Q-table 后发现 state 存在，其对应的四个 action 中，只有 action = 1 时的 value 最小，所以我们从剩下三个对应 value 最大的 action（0，2，3）中随机挑选一个 action，这里我们选择 action=2 作为 action  
$s$ = (0,0)，action $a$ = 2 后，玩家到达下一个 state $s\_$ = (0,1)为路径，所以收获的 reward 为$r$ = -1


In [18]:
a = 2
s = str(s)
s_ = str([0, 1])
r = -1

之后该$(s,a,s\_,r)$被用来更新 Q-table：
$$Q(s,a) = Q(s,a) + \alpha  (r + \gamma  \max_{a^{'}}Q(s\_ ,a^{'}) - Q(s,a) )$$
此时查询 Q-table，$Q(s,a)=0$, $\alpha$和$\gamma$都设为 0.9，查询$s\_$对应的 action 中最大的 value 值，发现$\max_{a^{'}} Q(s\_ ,a^{'}) = 3$,所以更新后的 Q(s,a) = 0 + 0.9 _ (-1 + 0.9 _ 3 - 0) = 0.9 \* 1.7 = 1.53


更新后的 Q-table：  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; action  
state&nbsp; &nbsp; &nbsp; 0 &nbsp; 1 &nbsp; 2 &nbsp; 3  
[0,0]&nbsp; &nbsp; &nbsp; 0 &nbsp; -1 1.53 0  
[0,1]&nbsp; &nbsp; &nbsp; 0 &nbsp; -1 &nbsp; 3 &nbsp; 2


In [19]:
q_table.loc[s, a] += alpha * (
    (r + gamma * q_table.loc[s_, :].max()) - q_table.loc[s, a]
)
print(q_table)

          0    1     2    3
[0, 0]  0.0 -1.0  1.53  0.0
[0, 1]  0.0 -1.0  3.00  2.0


**<font color = blue size=4>第二部分: tkinter 库介绍</font>**


Tkinter 是 Python 的标准 GUI 库。我们可以利用该库实现 python 图形界面的建立


In [20]:
import numpy as np
import time
import sys
import tkinter as tk

In [21]:
pixel_unit = 80  # 像素
maze_height = 4  # 迷宫高度
maze_weight = 4  # 迷宫宽度


class Grid_Paint(tk.Tk, object):
    def __init__(self):
        super(Grid_Paint, self).__init__()
        self.title("Grid_Paint")
        # 设置窗口的宽和高
        self.geometry(
            "{0}x{1}".format(maze_height * pixel_unit, maze_weight * pixel_unit)
        )
        # 调用构建迷宫函数来搭建迷宫
        self._build_maze()

    # 创造黑色的正方形方块
    # 输入分别为【一个网格的相对中心位置】，【障碍的横坐标】，【障碍的纵坐标】
    def creat_barrier(self, origin, x, y):
        # 计算出方块的中心位置
        center = origin + np.array([pixel_unit * x, pixel_unit * y])
        # 以该中心位置向四周进行黑色填充生成黑色方块
        self.barrier = self.canvas.create_rectangle(
            center[0] - 25, center[1] - 25, center[0] + 25, center[1] + 25, fill="black"
        )
        return self.barrier

    # 构建迷宫
    def _build_maze(self):
        # 画出白色背景
        self.canvas = tk.Canvas(
            self,
            bg="white",
            height=maze_height * pixel_unit,
            width=maze_weight * pixel_unit,
        )

        # 通过画线来构建网格
        for c in range(0, maze_weight * pixel_unit, pixel_unit):
            x0, y0, x1, y1 = c, 0, c, maze_height * pixel_unit
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, maze_height * pixel_unit, pixel_unit):
            x0, y0, x1, y1 = 0, r, maze_weight * pixel_unit, r
            self.canvas.create_line(x0, y0, x1, y1)

        # 每个网格的相对中心位置
        origin = np.array([int(0.5 * pixel_unit), int(0.5 * pixel_unit)])

        # （0，1）处创造黑色方块
        self.barrier1 = self.creat_barrier(origin, 0, 1)

        # （1，0）处创造红色圆形
        red_center = origin + np.array([pixel_unit * 1, pixel_unit * 0])
        self.oval = self.canvas.create_oval(
            red_center[0] - 25,
            red_center[1] - 25,
            red_center[0] + 25,
            red_center[1] + 25,
            fill="red",
        )

        # （0，0）处创造绿色正方形
        self.rect = self.canvas.create_rectangle(
            origin[0] - 25, origin[1] - 25, origin[0] + 25, origin[1] + 25, fill="green"
        )

        # （1，1）处创造粉色方块
        pink_center = origin + np.array([pixel_unit * 1, pixel_unit * 1])
        self.rect = self.canvas.create_rectangle(
            pink_center[0] - 25,
            pink_center[1] - 25,
            pink_center[0] + 25,
            pink_center[1] + 25,
            fill="pink",
        )

        # 打包所有元素
        self.canvas.pack()


# grid = Grid_Paint()
# grid.mainloop()

**<font color = blue size=4>第三部分: 实验任务</font>**


按照上面介绍的 Grid_Paint 类进行修改（也可以自己重新设计一个类）,同时实现一个 QLearning 类来完成具体方法，要求完成的任务要求包括:  
1.生成一个 6x6 的迷宫，其中包括起点，终点，障碍(至少 6 个)，陷阱(至少 1 个)；如果有其它设置也可以加入  
2.使用 Q-learning 算法在若干轮迭代（例如 100 轮）后找到一条从起点到终点的最短路径，得分规则自行设计，规则合理即可  
3.每轮迭代过程中需要在图形界面中显示玩家的移动过程，每次失败后更新 Q-table 表并重置玩家位置使得玩家能够继续在下一轮游戏中进行游戏  
4.最后输出玩家成功通关的次数和失败的次数，以及最后的 Q-table


下面为例子，帮助大家进行逻辑理解，仅作为参考，大家可以按照自己的思路完成，不强制要求按照下列逻辑完成


In [22]:
pixel_unit = 80  # 像素
maze_height = 6  # 迷宫高度
maze_weight = 6  # 迷宫宽度


class Grid_Paint(tk.Tk, object):
    def __init__(self):
        super(Grid_Paint, self).__init__()
        # 初始化动作指令
        self.action_space = ["u", "d", "l", "r"]
        # 方便之后用0，1，2，3代表上下左右
        self.n_actions = len(self.action_space)
        self.title("Grid_Paint")
        # 设置窗口的宽和高
        self.geometry(
            "{0}x{1}".format(maze_height * pixel_unit, maze_weight * pixel_unit)
        )
        # 调用构建迷宫函数来搭建迷宫
        self._build_maze()

    # 创造黑色的正方形方块
    # 输入分别为【一个网格的相对中心位置】，【障碍的横坐标】，【障碍的纵坐标】
    def creat_barrier(self, origin, x, y):
        # 计算出方块的中心位置
        center = origin + np.array([pixel_unit * x, pixel_unit * y])
        # 以该中心位置向四周进行黑色填充生成黑色方块
        self.barrier = self.canvas.create_rectangle(
            center[0] - 25, center[1] - 25, center[0] + 25, center[1] + 25, fill="black"
        )
        return self.barrier

    # 构建迷宫
    def _build_maze(self):
        # 画出白色背景
        self.canvas = tk.Canvas(
            self,
            bg="white",
            height=maze_height * pixel_unit,
            width=maze_weight * pixel_unit,
        )

        # 通过画线来构建网格
        for c in range(0, maze_weight * pixel_unit, pixel_unit):
            x0, y0, x1, y1 = c, 0, c, maze_height * pixel_unit
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, maze_height * pixel_unit, pixel_unit):
            x0, y0, x1, y1 = 0, r, maze_weight * pixel_unit, r
            self.canvas.create_line(x0, y0, x1, y1)

        # 每个网格的相对中心位置
        origin = np.array([int(0.5 * pixel_unit), int(0.5 * pixel_unit)])

        # 创造障碍  黑色方块
        self.barrier1 = self.creat_barrier(origin, 0, 5)
        self.barrier2 = self.creat_barrier(origin, 1, 1)
        self.barrier3 = self.creat_barrier(origin, 3, 4)
        self.barrier4 = self.creat_barrier(origin, 1, 5)
        self.barrier5 = self.creat_barrier(origin, 2, 3)
        self.barrier6 = self.creat_barrier(origin, 1, 2)

        # 创造陷阱 （2,2）处
        # 本人选择绿色方块作为陷阱
        trap_center = origin + np.array([pixel_unit * 2, pixel_unit * 2])
        self.trap = self.canvas.create_rectangle(
            trap_center[0] - 25,
            trap_center[1] - 25,
            trap_center[0] + 25,
            trap_center[1] + 25,
            fill="green",
        )

        # 迷宫中的实时位置
        # 本人选择红色圆作为实时位置
        red_center = origin + np.array([pixel_unit * 0, pixel_unit * 0])
        self.now = self.canvas.create_oval(
            red_center[0] - 25,
            red_center[1] - 25,
            red_center[0] + 25,
            red_center[1] + 25,
            fill="red",
        )

        # 创造终点
        # 本人选择蓝色方块
        pink_center = origin + np.array([pixel_unit * 5, pixel_unit * 5])
        self.end = self.canvas.create_rectangle(
            pink_center[0] - 25,
            pink_center[1] - 25,
            pink_center[0] + 25,
            pink_center[1] + 25,
            fill="blue",
        )

        # 打包所有元素
        self.canvas.pack()

    # 重置玩家位置，使得每一次游戏结束后玩家回到初始位置便于下一轮游戏，即回到起始位置
    def reset(self):
        self.update()
        time.sleep(0.5)
        self.canvas.delete(self.now)
        # 红色圆形
        origin = np.array([int(0.5 * pixel_unit), int(0.5 * pixel_unit)])
        self.now = self.canvas.create_oval(
            origin[0] - 25,
            origin[1] - 25,
            origin[0] + 25,
            origin[1] + 25,
            fill="red",
        )
        # 返回初始位置
        return self.canvas.coords(self.now)

    # 玩家移动，输入为移动指令
    def step(self, action):
        # 记录当前的state，也就是玩家现在的位置，s是一个1x4的数组，分别代表其左上角像素位置，右上角像素位置，左下角像素位置，右下角像素位置
        s = self.canvas.coords(self.now)
        base_action = np.array([0, 0])

        # 向上
        if action == 0:
            if s[1] > pixel_unit:
                base_action[1] -= pixel_unit
        # 向下
        elif action == 1:
            if s[1] < (maze_height - 1) * pixel_unit:
                base_action[1] += pixel_unit
        # 向右
        elif action == 2:
            if s[0] < (maze_weight - 1) * pixel_unit:
                base_action[0] += pixel_unit
        # 向左
        elif action == 3:
            if s[0] > pixel_unit:
                base_action[0] -= pixel_unit
        # 第一个参数是移动目标，第二个参数是到左上角的水平距离，第三个参数是距左上角的垂直距离。
        self.canvas.move(self.now, base_action[0], base_action[1])

        # 移动后的位置，也就是下一个state
        s_ = self.canvas.coords(self.now)

        # 将障碍物进行聚合
        barriers = [
            self.barrier1,
            self.barrier2,
            self.barrier3,
            self.barrier4,
            self.barrier5,
            self.barrier6,
        ]

        # reward判断
        # 如果碰到了陷阱，游戏结束
        if s_ == self.canvas.coords(self.trap):
            reward = -15
            status = True
            s_ = "died"

        # 如果碰到了障碍，游戏结束
        elif any(s_ == self.canvas.coords(barrier) for barrier in barriers):
            reward = -5
            status = True
            s_ = "died"

        # 如果到达了终点，则奖励为50，且游戏结束
        elif s_ == self.canvas.coords(self.end):
            reward = 50
            status = True
            s_ = "terminal"

        # 如果都没有碰到，则游戏继续，但是奖励为-1，代表移动的步数，否则无法去寻找最低步数
        else:
            reward = -1
            status = False

        # 返回state s在经过action之后的下一个state s_，获得的奖励 reward，以及此时游戏状态 status
        return s_, reward, status

    def render(self):
        time.sleep(0.1)
        self.update()

In [23]:
class QLearning:
    # 初始化，包括action列表，学习率，衰弱率，贪婪程度，以及Q-table
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.9):
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = pd.DataFrame(columns=self.actions, dtype=np.float64)

    # 选择action，输入参数为当前的state，输出的为在当前state的下一步action
    def choose_action(self, s):
        self.check_state_exist(s)
        # 首先判断该state在Q-table中是否存在，如果不存在则加入到Q-table
        # action 选择
        if np.random.uniform() < self.epsilon:
            # 贪婪模式
            # 挑选最佳的action
            state_action = self.q_table.loc[s, :]
            # 如果有多个action的value值都是最大，那就从这些中随机挑选
            a = np.random.choice(
                state_action[state_action == np.max(state_action)].index
            )
        else:
            # 非贪婪，探索模式
            # 随机挑选action
            a = np.random.choice(self.actions)
        return a

    # 学习以此不断更新Q-table，输入参数为一个state，做出的动作a，收获的奖励r，下一个state s_
    def learn(self, s, a, r, s_):
        # 首先判断下一个state s_在Q-table中是否存在，如果不存在则加入到Q-table
        self.check_state_exist(s_)

        # 先从Q-table中查询到Q(s,a)
        q_predict = self.q_table.loc[s, a]

        # 如果下一个state代表游戏结束，则不需要找下一个state s_能获得得最大value值，

        # 如果下一个state游戏继续，则首先找到下一个state s_能获得的最大value值
        if s_ != "terminal":
            q_target = r + self.gamma * self.q_table.loc[s_, :].max()
        else:
            q_target = r
        # 更新 Q-table
        self.q_table.loc[s, a] += self.alpha * (q_target - q_predict)

    # 检查state是否存在，输入为要检查的state
    def check_state_exist(self, s):
        if s not in self.q_table.index:
            # 将新状态添加到 Q-table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0] * len(self.actions),
                    index=self.q_table.columns,
                    name=s,
                )
            )


# 获取坐标
def set_state(s):
    loc = []
    loc.append(int((s[0] - maze_weight) / pixel_unit))
    loc.append(int((s[1] - maze_height) / pixel_unit))
    return loc

In [24]:
# 应用上面实现的两个类
grid = Grid_Paint()
RL = QLearning(actions=list(range(grid.n_actions)))
EPOCH = 100
right = 0
wrong = 0
for episode in range(EPOCH):
    print(episode)
    # 初始化玩家位置
    s = grid.reset()
    s = set_state(s)
    # 开始走迷宫
    while True:
        # 基于当前状态s确定下一步的action
        action = RL.choose_action(str(s))

        # 当前state在采取行动a后的下一个state s_,获得的奖励 r，当前游戏状态status
        s_, reward, status = grid.step(action)
        if s_ != "terminal" and s_ != "died":
            s_ = set_state(s_)  # 继续往下探索

        # 开始学习，更新Q-table
        # 输入参数为一个state，做出的动作a，收获的奖励r，下一个state s_
        RL.learn(str(s), action, reward, str(s_))
        q = RL.q_table.loc[str(s), action]

        # 将这一轮的s替换为s_,作为下一次的state s
        s = s_
        # 如果当前游戏状态结束，则跳出循环结束此次迭代，并根据本次结果记录是成功还是失败
        if status == True:
            right += 1
            s = grid.reset()
            s = set_state(s)
            break

# 输出结果
print("正确次数:", right)
print("错误次数:", 100-right)
print(RL.q_table)

0


C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(


1


C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(


2
3


...

C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(


12


C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
C:\Users\DELL\AppData\Local\Temp\ipykernel_26368\3980519262.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(


13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
正确次数: 100
错误次数: 0
                 0          1          2         3
[0, 0]   -3.366884  -3.319892  -3.333053 -3.363819
[1, 0]   -2.822695  -2.847664  -2.825867 -2.860123
[0, 1]   -2.913115  -2.873703  -2.847664 -2.894468
died      0.000000   0.000000   0.000000  0.000000
[2, 0]   -2.296823  -2.257152  -2.234564 -2.350546
[2, 1]   -1.769087  -2.850000  -1.749189 -2.047550
[0, 2]   -2.385032  -2.402620  -2.608516 -2.437979
[0, 3]   -1.919758  -1.956208  -1.954567 -1.970773
[3, 0]   -1.654862  -1.652852  -1.699633 -1.783956
[3, 1]   -1.312895  -1.253266  -0.454994 -1.309075
[4, 1]   -0.917469  -0.647522   4.139016 -0.962645
[4, 2]   -0.476453  -0.450100   3.477521 -0.480420
[3, 2]   -0.699628  -0.729093  -0.215283 -1.500000
[3, 3] 

: 

**<font color = blue size=4>第四部分:作业提交</font>**


一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个 ipynb 文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交


二、本次实验无需提交实验报告


三、课堂课件获取地址:https://www.jianguoyun.com/p/DaeV_a8Qp5WhChiIva4FIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DTMMhjMQp5WhChjTqK4FIAA
